## Summarize_webpage_via_Ollama

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!! 

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

This code demonstrates how to interact with the Ollama API to send a user prompt to a locally hosted AI model (llama3.2) and receive a response. It uses Python's requests library to perform the HTTP POST request.



In [ ]:
# Constants

OLLAMA_API = "your_api_key"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json())  # This will print the entire JSON response to check its structure

{'model': 'llama3.2', 'created_at': '2024-11-25T10:17:45.5767145Z', 'message': {'role': 'assistant', 'content': "Generative AI has numerous business applications across various industries. Here are some examples:\n\n1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses reduce the time and cost associated with creating content from scratch.\n2. **Image and Video Generation**: Generative AI can be used to create visually appealing images and videos for marketing campaigns, product demonstrations, and more. This can help businesses reduce their reliance on traditional graphic design and video production methods.\n3. **Chatbots and Customer Service**: Generative AI can be used to power chatbots that provide 24/7 customer support, answering frequently asked questions, and helping customers with simple inquiries.\n4. **Predictive Maintenance**: Generative AI can be use

## Developing Web Page

In [ ]:
# A class to represent a Webpage


class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
cnn = Website("https://edition.cnn.com/")
print(cnn.title)
print(cnn.text)

Breaking News, Latest News and Videos | CNN
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
More
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Watch
Listen
Live TV
Subscribe
Sign in
My Account
Settings
Topics You Follow
Sign Out
Your C

In [ ]:
# Define our system prompt 

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [49]:
print(user_prompt_for(cnn))

You are looking at a website titled Breaking News, Latest News and Videos | CNN
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
More
US
World
Politi

In [54]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [55]:
messages_for(cnn)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Breaking News, Latest News and Videos | CNN\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nCNN values your feedback\n1. How relevant is this ad to you?\n2. Did you encounter any technical issues?\nVideo player was slow to load content\nVideo content never loaded\nAd froze or did not finish loading\nVideo content did not start after ad\nAudio on ad was too loud\nOther issues\nAd never loaded\nAd prevented/slowed the page from loading\nContent moved around while ad loaded\nAd was repetitive to ads I've seen previously\nOther issues\nCancel\nSubmit\nThank You!\nYour effort and contribution in providing thi

In [56]:
def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }
    response = ollama.chat(model=MODEL, messages=messages)
    print(response['message']['content'])

In [57]:
# Example usage:
url = "https://edition.cnn.com/"
print(summarize(url))

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as blog posts, articles, social media posts, and product descriptions, saving time and resources for content creators.
2. **Product Design**: Generative AI can design new products, such as furniture, architecture, or even entire buildings, using parametric modeling and generative algorithms.
3. **Marketing Automation**: AI-powered marketing tools can generate personalized marketing campaigns, social media ads, and email newsletters, increasing the effectiveness of marketing efforts.
4. **Customer Service Chatbots**: Generative AI can create conversational chatbots that respond to customer inquiries, providing 24/7 support and improving customer satisfaction.
5. **Creative Writing and Storytelling**: AI can generate creative content such as stories, poems, or even entire scripts for films and TV shows, augmenting the work of human w